In [ ]:
import dalmatian as dm

In [ ]:
# First part replaces the old files with outputs from David's rerun w/ updated star and rsem indices
# First make sure new star has been run on all samples
wm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_hg38_RNAseq").disable_hound()
rna_samples = wm.get_samples()
rna_samples

In [ ]:
t = tracker.getTracker()

In [ ]:
# Make sure all rna samples on terra are on the tracker
rna_ids = set(rna_samples.index)
tracker_ids = set(t.index)
rna_ids - tracker_ids

In [ ]:
rna_samples.david_internal_bam_filepath.isna().sum()

In [ ]:
for i in rna_ids:
    fn = "gs://cclebams/rnasq_hg38/" + i + ".Aligned.sortedByCoord.out.bam"
    if t.loc[i, "internal_bam_filepath"] != fn:
        print(i)

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
proteincoding_genes_tpm_logp1 = tc.get(name='expression-d035', version=21, file='proteincoding_genes_tpm_logp1')

In [ ]:
'ACH-001767' in proteincoding_genes_tpm_logp1.index

In [ ]:
for i in rna_ids:
    if t.loc[i, "internal_bam_filepath"] != rna_samples.loc[i, "internal_bam_filepath"]:
        print(i)

In [ ]:
from genepy.utils import helper as h

to_move = []
for i in rna_samples.index:
    david_bam = rna_samples.loc[i, 'david_internal_bam_filepath']
    david_bai = rna_samples.loc[i, 'david_internal_bai_filepath']
    old_bam = rna_samples.loc[i, "internal_bam_filepath"]
    old_bai = rna_samples.loc[i, "internal_bai_filepath"]
    
    to_move.append((david_bam, old_bam))
    to_move.append((david_bai, old_bai))
to_move

In [ ]:
# test on one sample
# h.parrun(['gsutil -u broad-firecloud-ccle mv gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/96d2add9-c915-4d1b-8c67-afe2d3f349ec/star_workflow/39f9cd7c-3ed6-48b8-94b3-e95f9ee4673b/call-star/star_out/CDS-ABH0uZ.Aligned.sortedByCoord.out.bam gs://cclebams/rnasq_hg38/CDS-ABH0uZ.Aligned.sortedByCoord.out.bam'], cores=20)
for (i, v) in to_move[:6]:
    print(i + " moving to " + v)

In [ ]:
h.parrun(['gsutil -u broad-firecloud-ccle mv ' + i + ' ' + v for (i, v) in to_move[6:]], cores=20)

In [ ]:
rna_samples.to_csv('rna_terraws_backup.csv')

In [ ]:
cols_to_delete = ['david_more_resources_rsem_genes',
    'david_more_resources_rsem_isoforms',
    'rsem_genes',
    'rsem_isoforms',
    'star_chimeric_bam_file',
    'star_chimeric_bam_index',
    'star_chimeric_junctions',
    'star_junctions',
    'star_junctions_pass1',
    'star_logs',
    'star_read_counts',
    'star_transcriptome_bam']

In [ ]:
wm.delete_sample_attributes(rna_samples[cols_to_delete], delete_files=True, dry_run=False)

In [ ]:
rna_samples = wm.get_samples()
rna_samples

In [ ]:
renaming_dict = {'david_internal_bai_filepath': 'internal_bai_filepath',
    'david_internal_bam_filepath': 'internal_bam_filepath',
    'david_rsem_genes': 'rsem_genes',
    'david_rsem_isoforms': 'rsem_isoforms',
    'david_star_chimeric_bam_file': 'star_chimeric_bam_file',
    'david_star_chimeric_bam_index': 'star_chimeric_bam_index',
    'david_star_chimeric_junctions': 'star_chimeric_junctions',
    'david_star_junctions': 'star_junctions',
    'david_star_junctions_pass1': 'star_junctions_pass1',
    'david_star_logs': 'star_logs',
    'david_star_read_counts': 'star_read_counts',
    'david_star_transcriptome_bam': 'star_transcriptome_bam',
                }

In [ ]:
rna_samples = rna_samples.rename(columns=renaming_dict)
rna_samples

In [ ]:
rna_samples.columns

In [ ]:
wm.update_sample_attributes(rna_samples)

In [ ]:
rna_samples = wm.get_samples()
rna_samples

In [ ]:
renaming_dict.keys()

In [ ]:
wm.delete_sample_attributes(rna_samples[renaming_dict.keys()], delete_files=False, dry_run=False)

In [ ]:
rna_samples = wm.get_samples()

In [ ]:
# some lines are missing rsem run outputs, rerun on them:
rerun_rsem = rna_samples[rna_samples.rsem_genes.isnull()]
rerun_rsem.index

In [ ]:
wm.update_sample_set(sample_set_id="rerun_rsem", sample_ids=rerun_rsem.index)

In [ ]:
rna_samples.star_chimeric_junctions.isnull().sum()

In [ ]:
# Second part!! several months later!! 
# where we updated STAR and RSEM indices AGAIN using the ref fasta from here: https://github.com/broadinstitute/gtex-pipeline/blob/master/TOPMed_RNAseq_pipeline.md
# and did a remapping in the dev workspace. Now we move things to the main processing workspace

In [ ]:
wm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_hg38_RNAseq").disable_hound()
wm_dev = dm.WorkspaceManager("broad-firecloud-ccle/DEV_DepMap_hg38_RNAseq").disable_hound()

In [ ]:
dev_samples = wm_dev.get_samples()
dev_ids = set(dev_samples.index)
rna_samples = wm.get_samples()
rna_ids = set(rna_samples.index)

for i in dev_ids:
    fn = "gs://cclebams/rnasq_hg38/" + i + ".Aligned.sortedByCoord.out.bam"
    if rna_samples.loc[i, "internal_bam_filepath"] != fn:
        rna_samples.loc[i, "internal_bam_filepath"] = fn
        print(fn)

In [ ]:
from genepy.utils import helper as h

to_move = []
for i in dev_samples.index:
    new_bam = dev_samples.loc[i, 'internal_bam_filepath']
    new_bai = dev_samples.loc[i, 'internal_bai_filepath']
    new_transcriptome = dev_samples.loc[i, 'star_transcriptome_bam']
    old_bam = "gs://cclebams/rnasq_hg38/" + i + ".Aligned.sortedByCoord.out.bam"
    old_bai = "gs://cclebams/rnasq_hg38/" + i + ".Aligned.sortedByCoord.out.bam.bai"
    old_transcriptome = rna_samples.loc[i, 'star_transcriptome_bam']
    
    to_move.append((new_bam, old_bam))
    to_move.append((new_bai, old_bai))
    to_move.append((new_transcriptome, old_transcriptome))

In [ ]:
for (i, v) in to_move[:6]:
    print(i + " moving to " + v)

In [ ]:
h.parrun(['gsutil -u broad-firecloud-ccle mv ' + i + ' ' + v for (i, v) in to_move[:6]], cores=20)

In [ ]:
h.parrun(['gsutil -u broad-firecloud-ccle mv ' + i + ' ' + v for (i, v) in to_move[6:]], cores=20)

In [ ]:
# other output columns from RNA_pipeline
cols = ['star_chimeric_bam_file', 'star_chimeric_junctions', 'rnaseqc2_exon_counts', 'fusion_predictions', 
        'rnaseqc2_gene_counts', 'rnaseqc2_gene_tpm', 'rnaseqc2_insertsize_distr', 'star_junctions', 
        'star_junctions_pass1', 'rnaseqc2_metrics', 'star_read_counts']
cols_important = ['fusion_predictions_abridged', 'rsem_genes', 'rsem_isoforms']

In [ ]:
from genepy import terra
from genepy.google import gcp
from genepy.utils import helper as h
import dalmatian as dm

wmto = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_hg38_RNAseq")

for c in cols:
    print(c)
    res, _, cmds = terra.changeGSlocation(
            "broad-firecloud-ccle/DEV_DepMap_hg38_RNAseq",
            newgs="gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/",
            bamfilepaths=[c],
            entity="sample",
            keeppath=True,
            dry_run=True,
            workspaceto="broad-firecloud-ccle/DepMap_hg38_RNAseq",
        )
#     fns = [cmd.split()[2] for cmd in cmds]
#     valid = set(gcp.lsFiles(fns))
#     valid_cmd = [c for c in cmds if c.split()[2] in valid]
#     print(len(valid))
#     h.parrun(valid, cores=20)
    
    print("updating column " + c + " in the terra workspace")
    wmto.disable_hound().update_entity_attributes("sample", res)


In [ ]:
wmto = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_hg38_RNAseq")
#wmto.disable_hound().update_entity_attributes("sample", res)

In [ ]:
wmfrom = dm.WorkspaceManager("broad-firecloud-ccle/DEV_DepMap_hg38_RNAseq")

sfrom = wmfrom.get_samples()
sto = wmto.get_samples()

wmfrom.disable_hound().update_entity_attributes("sample", sto.loc[set(sfrom.index), ['internal_bam_filepath', 'internal_bai_filepath', 'star_transcriptome_bam']])

In [ ]:
sfrom = wmfrom.get_samples()
sto = wmto.get_samples()
sto.loc[set(sfrom.index), cols + cols_important]

In [ ]:
for k, v in res.items():
    print(k, len(v))